In [2]:
import skimage.io as io
import matplotlib.pyplot as plt
from scipy import ndimage
import os
import random as r
import cv2
import numpy as np
from glob import glob
from numba import jit, cuda

In [7]:
@jit
def seg_to_array(path, end, label):
    # get locations
    files = glob(path+end, recursive=True)
    img_list = []
    
    r.seed(42)
    r.shuffle(files)
    
    for file in files:
        img = io.imread(file, plugin="simpleitk")
        # all tumor
        if label == 1:
            img[img != 0] = 1
        # Non-enhancing Tumor
        if label == 2:
            img[img != 1] = 0
        # Without Edema
        if label == 3:
            img[img == 2] = 0
            img[img != 0] = 1
        # Enhancing Tumor
        if label == 4:
            img[img != 4] = 0
            img[img == 4] = 1
        img.astype("float32")
        for slice in range(60, 130):
            img_s = img[slice,:,:]
            # resize
            img_s = cv2.resize(img_s, (128,128))
            img_s = np.expand_dims(img_s, axis=0)
            img_list.append(img_s)
            
    return np.array(img_list,np.float32)

@jit
def to_array(path, end):
    # get locations
    files = glob(path+end, recursive=True)
    img_list = []
    
    r.seed(42)
    r.shuffle(files)
    
    for file in files:
        img = io.imread(file, plugin="simpleitk")
        # standardization
        img = (img-img.mean())/img.std()
        img.astype("float32")
        
        for slice in range(60, 130):
            img_s = img[slice,:,:]
            # resize
            img_s = cv2.resize(img_s, (128,128))
            img_s = np.expand_dims(img_s, axis=0)
            img_list.append(img_s)
            
    return np.array(img_list,np.float32)



In [8]:
def mostrar_img(img_flair,img_t2,img_t1,img_t1ce,img_completo_004,img_seg,plano,seg):
    a=c=i=plano
    s=seg
    print(f"profundidad {i}")
    print(f"profundidad segmentación {s}")
    fig, (axs1,axs2,axs3,axs4,axs5,axs6) = plt.subplots(6, 3, dpi=180, figsize=(20, 30))

    axs1[0].imshow(ndimage.rotate(img_flair[i,:,:], 180), cmap='gray')
    axs1[0].set_title("Plano axial")
    axs1[1].imshow(ndimage.rotate(img_flair[:,i,:], 180), cmap='gray')
    axs1[1].set_title("Plano coronal")
    axs1[2].imshow(ndimage.rotate(img_flair[:,:,s], 180), cmap='gray')
    axs1[2].set_title("Plano sagital");

    axs2[0].set_title("T2")
    axs2[0].imshow(ndimage.rotate(img_t2[i,:,:], 180), cmap='gray')
    axs2[1].imshow(ndimage.rotate(img_t2[:,i,:], 180), cmap='gray')
    axs2[2].imshow(ndimage.rotate(img_t2[:,:,s], 180), cmap='gray')

    axs3[0].set_title("T1")
    axs3[0].imshow(ndimage.rotate(img_t1[i,:,:], 180), cmap='gray')
    axs3[1].imshow(ndimage.rotate(img_t1[:,i,:], 180), cmap='gray')
    axs3[2].imshow(ndimage.rotate(img_t1[:,:,s], 180), cmap='gray')

    axs4[0].set_title("T1ce")
    axs4[0].imshow(ndimage.rotate(img_t1ce[i,:,:], 180), cmap='gray')
    axs4[1].imshow(ndimage.rotate(img_t1ce[:,i,:], 180), cmap='gray')
    axs4[2].imshow(ndimage.rotate(img_t1ce[:,:,s], 180), cmap='gray')

    axs5[0].set_title("las 4 secuencias en un solo arreglo")
    axs5[0].imshow(ndimage.rotate(img_completo_004[i,:,:,:].astype('uint8'), 180))
    axs5[1].imshow(ndimage.rotate(img_completo_004[:,i,:,:].astype('uint8'), 180))
    axs5[2].imshow(ndimage.rotate(img_completo_004[:,:,s,:].astype('uint8'), 180))

    axs6[0].set_title("Segmentación de imágenes")
    axs6[0].imshow(ndimage.rotate(img_seg[i,:,:], 180))
    axs6[1].imshow(ndimage.rotate(img_seg[:,i,:], 180))
    axs6[2].imshow(ndimage.rotate(img_seg[:,:,s], 180))
    
    return

In [4]:
def mostrar_img2(img_completo_004,c,color="gray"):

    i=c
    # Visualizamos las distintas secuencias
    print(f"profundidad {i}")
    plt.figure(1,figsize=(20,24))
    plt.subplot(1, 4, 1)
    plt.imshow(ndimage.rotate(img_completo_004[i,:,:,0],1),cmap = color)
    plt.title('flair')

    plt.subplot(1, 4, 2)
    plt.imshow(ndimage.rotate(img_completo_004[i,:,:,1],1),cmap=color)
    plt.title('t1ce')

    plt.subplot(1, 4, 3)
    plt.imshow(ndimage.rotate(img_completo_004[i,:,:,2],1),cmap=color)
    plt.title('t1')

    plt.subplot(1, 4, 4)
    plt.imshow(ndimage.rotate(img_completo_004[i,:,:,3],1),cmap=color)
    plt.title('t2')
    plt.show()
    i +=1 
    
    return

In [8]:
def mostrar_img_entradas(img_flair,img_t2,img_seg,pla,seg,color="gray"):

    i=seg
    s=pla
    seg_all = img_seg.copy()
    seg_all[seg_all != 0] = 1 # tumor completo
    # Visualizamos las distintas secuencias
    print(f"profundidad {i}")
    plt.figure(1,figsize=(20,24))
    plt.subplot(1, 4, 1)
    plt.imshow(ndimage.rotate(img_flair[i,:,:],1),cmap = color)
    plt.title('flair')

    plt.subplot(1, 4, 2)
    plt.imshow(ndimage.rotate(img_t2[i,:,:],1),cmap=color)
    plt.title('t2')

    plt.subplot(1, 4, 3)
    plt.imshow(ndimage.rotate(seg_all[s,:,:],1),cmap=color)
    plt.title('segmentación')

  
    i +=1 
    
    return

In [10]:
def mostrar_seg(img_seg,img_completo_004,pla,seg):
    i=seg
    s=pla
    print(f"profundidad {s}")
    print(f"profundidad segmentación {i}")
    fig = plt.figure(figsize=(30,25))
    seg_all = img_seg.copy()
    seg_all[seg_all != 0] = 1 # tumor completo
    seg_non = img_seg.copy()
    seg_non[seg_non != 1] = 0 # tumor completo
    seg_real = img_seg.copy()
    seg_real[seg_real != 4] = 0 # tumor completo
    seg_edema = img_seg.copy()
    seg_edema[seg_edema == 1] = 0 # Sin realce
    seg_edema[seg_edema == 4] = 0 # Con realce
    seg_edema[seg_edema != 0] = 1 # Edema

    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(ndimage.rotate(img_seg[:,:,i],180))
    plt.title("Segmentación de tumor", fontsize=15);

    ax2 = fig.add_subplot(1,3,2)
    ax2.imshow(ndimage.rotate(seg_all[:,:,i],180))
    plt.title("Área de tumor completa", fontsize=15);

    ax3 = fig.add_subplot(1,3,3)
    ax3.imshow(ndimage.rotate(seg_non[:,:,i],180))
    plt.title("Tumor sin realce", fontsize=15);

    ax4 = fig.add_subplot(2,3,4)
    ax4.imshow(ndimage.rotate(seg_real[:,:,i],180))
    plt.title("Tumor con realce", fontsize=15);

    ax5 = fig.add_subplot(2,3,5)
    ax5.imshow(ndimage.rotate(seg_edema[:,:,i],180))
    plt.title("Edema", fontsize=15);
    
    ax6 = fig.add_subplot(2,3,6)
    ax6.imshow(ndimage.rotate(img_completo_004[:,:,s,3],180))
    plt.title("Completo", fontsize=15);
    return

In [5]:
def mostrar_seg_planos(img_seg,img_completo_004,pla,seg):
    i=seg
    s=pla
    # img_seg,img_completo_004,155,155

    print(f"profundidad {s}")
    print(f"profundidad segmentación {i}")
    # fig = plt.figure(figsize=(30,25))
    seg_all = img_seg.copy()
    seg_all[seg_all != 0] = 1 # tumor completo
    seg_non = img_seg.copy()
    seg_non[seg_non != 1] = 0 # tumor completo
    seg_real = img_seg.copy()
    seg_real[seg_real != 4] = 0 # tumor completo
    seg_edema = img_seg.copy()
    seg_edema[seg_edema == 1] = 0 # Sin realce
    seg_edema[seg_edema == 4] = 0 # Con realce
    seg_edema[seg_edema != 0] = 1 # Edema


    fig, (axs0,axs1,axs2,axs3,axs4,axs5) = plt.subplots(6, 3, dpi=180, figsize=(20, 30))

    axs0[0].set_title("Plano axial")
    axs0[0].imshow(ndimage.rotate(img_completo_004[i,:,:,3], 1), cmap='gray')
    axs0[1].set_title("Plano coronal")
    axs0[1].imshow(ndimage.rotate(img_completo_004[:,i,:,3], 180), cmap='gray')
    axs0[2].set_title("Plano sagital");
    axs0[2].imshow(ndimage.rotate(img_completo_004[:,:,s,3], 180), cmap='gray')

    axs1[0].imshow(ndimage.rotate(img_seg[i,:,:], 1))
    axs1[0].set_title("Segmentación de tumor")
    axs1[1].imshow(ndimage.rotate(img_seg[:,i,:], 180))
    axs1[2].imshow(ndimage.rotate(img_seg[:,:,s], 180))

    axs2[0].set_title("Área de tumor completa")
    axs2[0].imshow(ndimage.rotate(seg_all[i,:,:], 1))
    axs2[1].imshow(ndimage.rotate(seg_all[:,i,:], 180))
    axs2[2].imshow(ndimage.rotate(seg_all[:,:,s], 180))

    axs3[0].set_title("Tumor sin realce")
    axs3[0].imshow(ndimage.rotate(seg_non[i,:,:], 1))
    axs3[1].imshow(ndimage.rotate(seg_non[:,i,:], 180))
    axs3[2].imshow(ndimage.rotate(seg_non[:,:,s], 180))

    axs4[0].set_title("Tumor con realce")
    axs4[0].imshow(ndimage.rotate(seg_real[i,:,:], 1))
    axs4[1].imshow(ndimage.rotate(seg_real[:,i,:], 180))
    axs4[2].imshow(ndimage.rotate(seg_real[:,:,s], 180))

    axs5[0].set_title("Edema")
    axs5[0].imshow(ndimage.rotate(seg_edema[i,:,:].astype('uint8'), 1))
    axs5[1].imshow(ndimage.rotate(seg_edema[:,i,:].astype('uint8'), 180))
    axs5[2].imshow(ndimage.rotate(seg_edema[:,:,s].astype('uint8'), 180))
    return
    


In [5]:
def grafico_hist(history):
    fig = plt.gcf()
    fig.set_size_inches(12, 8)

    plt.subplot(2,2,1)
    plt.title('Pérdida')
    plt.xlabel("epoch")
    plt.plot(history['loss'], color='red', label='train')
    plt.plot(history['val_loss'],color ='blue',label='validation')
    plt.legend(['entrenamiento', 'validación'], loc='upper right')

    plt.subplot(2,2,2)
    plt.title('Precisión')
    plt.xlabel("epoch")
    plt.plot(history['accuracy'], color='red', label='train')
    plt.plot(history['val_accuracy'], color='blue', label='validation')
    plt.legend(['entrenamiento', 'validación'], loc='lower right')
    plt.show()
    
    return

In [6]:
def vcrep_kpart(X, y, k=5):
    n = X.shape[0]
    t_pliegue = int(n / k)
    perm = np.random.permutation(n)
    X = X[perm]
    y = y[perm]
    masc_val = np.zeros(n, dtype=bool)
    masc_val[0:t_pliegue] = True
    masc_ent = np.logical_not(masc_val)
    X_ent = X[masc_ent]
    y_ent = y[masc_ent]
    X_val = X[masc_val]
    y_val = y[masc_val]

    return X_ent, y_ent, X_val, y_val